In [48]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv
import random
import lxml
import pandas as pd
from bs4 import BeautifulSoup

In [50]:
print "importing libraries..."

url = 'https://therapists.psychologytoday.com/rms/county/NY/New+York.html'
driver = webdriver.Chrome()
driver.get(url)
soup = BeautifulSoup(driver.page_source,'lxml')
#main_table = BeautifulSoup(.content, 'html5lib').findAll("div",{"class":"no-min-height"})[0]

main_table = soup.findAll('div',{'class':'no-min-height'})[0]

next_page_url = main_table.contents[2].contents[7].contents[1].contents[3].contents[1].find_all('a',href=True)[~0]['href']

importing libraries...


In [39]:
#grabs surface content
docs = [main_table.contents[2].contents[5].contents[1].contents[i].contents[1].contents[1].attrs for i in range(1,42,2)]

In [46]:
pd.DataFrame(docs).drop(['class','data-new-clients','style'],axis=1).dropna(axis=0)

,data-email-url,data-phone,data-prof-name,data-profid,data-profile-image,data-profile-url,data-result-row,data-result-url,data-use-company-name,itemscope,itemtype
0,https://therapists.psychologytoday.com/rms/pro...,(917) 521-6329,Theo Tsaousides,154435,https://cdn0.sussexdirectories.com/rms/rms_pho...,https://therapists.psychologytoday.com/rms/pro...,1,https://therapists.psychologytoday.com/rms/pro...,,,http://schema.org/Person
1,https://therapists.psychologytoday.com/rms/pro...,(646) 600-8104,Gene Lubow,56339,https://cdn2.sussexdirectories.com/rms/rms_pho...,https://therapists.psychologytoday.com/rms/pro...,2,https://therapists.psychologytoday.com/rms/pro...,,,http://schema.org/Person
2,https://therapists.psychologytoday.com/rms/pro...,(646) 666-3537,Rebecca A Harmon,292249,https://cdn4.sussexdirectories.com/rms/rms_pho...,https://therapists.psychologytoday.com/rms/pro...,3,https://therapists.psychologytoday.com/rms/pro...,,,http://schema.org/Person
3,https://therapists.psychologytoday.com/rms/pro...,(646) 490-9989,Flavio Casoy,244513,https://cdn0.sussexdirectories.com/rms/rms_pho...,https://therapists.psychologytoday.com/rms/pro...,4,https://therapists.psychologytoday.com/rms/pro...,,,http://schema.org/Person
4,https://therapists.psychologytoday.com/rms/pro...,(646) 495-9842,Griffin Hansbury,115091,https://cdn2.sussexdirectories.com/rms/rms_pho...,https://therapists.psychologytoday.com/rms/pro...,5,https://therapists.psychologytoday.com/rms/pro...,,,http://schema.org/Person
5,https://therapists.psychologytoday.com/rms/pro...,(516) 629-5972,Henry Gonzalez,180139,https://cdn4.sussexdirectories.com/rms/rms_pho...,https://therapists.psychologytoday.com/rms/pro...,6,https://therapists.psychologytoday.com/rms/pro...,,,http://schema.org/Person
6,https://therapists.psychologytoday.com/rms/pro...,(646) 600-5082,Jonathan Eger,69138,https://cdn0.sussexdirectories.com/rms/rms_pho...,https://therapists.psychologytoday.com/rms/pro...,7,https://therapists.psychologytoday.com/rms/pro...,,,http://schema.org/Person
7,https://therapists.psychologytoday.com/rms/pro...,(516) 589-7754,Jane F. Lane,332612,https://cdn2.sussexdirectories.com/rms/rms_pho...,https://therapists.psychologytoday.com/rms/pro...,8,https://therapists.psychologytoday.com/rms/pro...,,,http://schema.org/Person
8,https://therapists.psychologytoday.com/rms/pro...,(347) 897-0315,H. Lou Himes,153096,https://cdn4.sussexdirectories.com/rms/rms_pho...,https://therapists.psychologytoday.com/rms/pro...,9,https://therapists.psychologytoday.com/rms/pro...,,,http://schema.org/Person
10,https://therapists.psychologytoday.com/rms/pro...,(941) 787-5379,"Beatty Cohan, MSW, LCSW, AASECT",60144,https://cdn0.sussexdirectories.com/rms/rms_pho...,https://therapists.psychologytoday.com/rms/pro...,10,https://therapists.psychologytoday.com/rms/pro...,1,,http://schema.org/Organization


In [26]:
main_table.contents[2].contents[5].contents[1]

<div class="col-xs-12 col-sm-9 col-sm-push-3 col-md-10 col-md-push-2 col-lg-10 col-lg-push-2">\n<div class="row">\n<div class="col-xs-12 col-sm-12 col-md-12 col-lg-12">\n<div class="result-row normal-result" data-email-url="https://therapists.psychologytoday.com/rms/profile/154435/contact" data-new-clients="1" data-phone="(917) 521-6329" data-prof-name="Theo Tsaousides" data-profid="154435" data-profile-image="https://cdn0.sussexdirectories.com/rms/rms_photos/sized/35/44/154435-511238-2_120x150.jpg?pu=1449725330" data-profile-url="https://therapists.psychologytoday.com/rms/prof_detail.php?profid=154435" data-result-row="1" data-result-url="https://therapists.psychologytoday.com/rms/prof_detail.php?profid=154435&amp;ref=1&amp;sid=1500570866.8625_25819&amp;county=New+York&amp;state=NY" data-use-company-name="" itemscope="" itemtype="http://schema.org/Person" style="position: relative;">\n<div class="row">\n<div class="col-xs-4 col-sm-2 col-md-2 col-lg-2">\n<a href="https://therapists.psy